# test_vertical_interpolation

see interpolate_f1_to_isobaric.ipynb


# last edited 12 October 2018

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [9]:
print('BEGIN')

%matplotlib inline

__file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

import numpy as np
import numpy.ma as ma
import netCDF4
import inspect
# import pickle
# import bz2
import glob
import socket
import re
import os
import timeit
import subprocess
# import scipy.stats as st
# import matplotlib.pyplot as plt
# import math
# from scipy import stats
# import pandas as pd
# from matplotlib.ticker import ScalarFormatter, FormatStrFormatter, FixedLocator
import sys
import shlex
from subprocess import Popen, PIPE
import shutil

def get_exitcode_stdout_stderr(cmd):
    """
    Execute the external command and get its exitcode, stdout and stderr.
    """
    args = shlex.split(cmd)
 
    proc = Popen(args, stdout=PIPE, stderr=PIPE)
    out, err = proc.communicate()
    exitcode = proc.returncode
    #
    return exitcode, out, err

CRED = '\033[91m'
CEND = '\033[0m'

hostname=socket.gethostname()

print('hostname=',hostname)

if(re.match('raijin',hostname)):
  print('this is rajin')
  rundir='/short/v14/mac599/cafepp/rundir'
  topdir=''
elif(re.match('oa-3.-cdc',hostname)):
  print('this is vm32')
  rundir='/OSM/CBR/OA_DCFP/work/col414/cafepp'
  topdir='/OSM/CBR/OA_DCFP/data/CAFEPP/CMIP6'

os.chdir(rundir)

os.chdir('/OSM/CBR/OA_DCFP/work/col414/cafepp')

print('Current Working Directory=',os.getcwd())

from n_data_funcs import n_data_funcs

from decadal_diag import \
  convert_bytes, \
  file_size, \
  compress_nc
  
#   box_indices, \
#   diag_isothetaoNc, \
#   calc_isoN, \
#   shade_2d_latlon, \
#   shade_2d_simple, \
#   smooth, \
#   plot_xy, \
#   plot_box_indices, \
#   nino_indices, \
#   plot_2d_scatter, \
#   lagcorr, \
#   file_sort_ripf, \
#   file_spec_summary, \
#   basic_stats

print('END')

BEGIN
hostname= oa-35-cdc
this is vm32
Current Working Directory= /OSM/CBR/OA_DCFP/work/col414/cafepp
END


In [ ]:
# print('BEGIN')

# input_files=sorted(glob.glob('/OSM/CBR/OA_DCFP/data/intermediate_products/squ027/f1_isobaric/yr2003/mn1/OUTPUT.1/atmos_daily_????_01_01.plevel.nc'))

# ifhN=netCDF4.MFDataset(input_files)

# print('END')

In [24]:
def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0


def file_size(file_path):
    """
    this function will return the file size
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)
      
def prepare_vertintp(input_data_file, input_grid_file, output_file, **kwargs):
  '''
  Copy the entire contents of a file and apply compression to output variables.
  Note all options & combinations tested.
  
  If Diag=True, put it first in the list of option/arguments.

  prepare_vertintp(input_data_file, input_grid_file, output_filet, Diag=True/False, nc_model=['NETCDF4_CLASSIC',...], compresssion=[1...9], history=True/False, Clobber=True/False)
  
  '''
  
  import numpy as np
  import netCDF4
  import inspect
  import os
  import timeit
  import datetime

  Diag=zlib=Clobber=False
  history=True
  nc_model='NETCDF4_CLASSIC'

  #print('kwargs.items()=',kwargs.items())

  for key, value in kwargs.items():
    if(key=='Diag'):
      if(str(value)=='True'):
        Diag=True
      elif(str(value)=='False'):
        Diag=False
      else:
        raise SystemExit('Diag either True or False:'+__file__+' line number: '+str(inspect.stack()[0][2]))
      if(Diag): print('Turning on diagnostics.')
    elif(key=='nc_model'):
      nc_model=value
      if(Diag): print('Using nc model=',nc_model)
    elif(key=='compression'):
      zlib=True
      complevel=int(value)
      if(Diag and zlib): print('Compressing with level (b/w 1-9)=',complevel)
    elif(key=='history'):
      if(str(value)=='True'):
        history=True
      elif(str(value)=='False'):
        history=False
      else:
        raise SystemExit('history either True or False:'+__file__+' line number: '+str(inspect.stack()[0][2]))
      if(Diag and history): print('global history attribute being appended to or created.')
    elif(key=='Clobber'):
      if(str(value)=='True'):
        Clobber=True
      elif(str(value)=='False'):
        Clobber=False
      else:
        raise SystemExit('Clobber either True or False:'+__file__+' line number: '+str(inspect.stack()[0][2]))
      if(Diag and Clobber): print('Overwriting output file if it exists.')
    
  #print('input grid file=',input_grid_file)
  
  if(not os.path.exists(input_data_file)):
    raise SystemExit('input data file, '+input_data_file+', doesnt exist:'+__file__+' line number: '+str(inspect.stack()[0][2]))

  if(type(input_grid_file)!=type(None) and not os.path.exists(input_grid_file)):
    raise SystemExit('input grid file, '+input_grid_file+', doesnt exist:'+__file__+' line number: '+str(inspect.stack()[0][2]))
    
  #print('Clobber=',Clobber)
  #print('type(Clobber)=',type(Clobber))

  #if(Diag): time_start = timeit.timeit()
  
  input_data_file_size=file_size(input_data_file)
  
  if(type(input_grid_file)!=type(None)):
    input_grid_file_size=file_size(input_grid_file)
  
  print('Input data file: '+input_data_file+', File size='+input_data_file_size)
  if(type(input_grid_file)!=type(None)):
    print('Input grid file: '+input_grid_file+', File size='+input_grid_file_size)
  print('Output file: '+output_file)
  
  if(not Clobber and os.path.exists(output_file)):
    raise SystemExit('not Clobber and output exists:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
  if(Clobber and os.path.exists(output_file)):
    os.remove(output_file)

  #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
  if(type(input_grid_file)!=type(None)):
    ifh_grid = netCDF4.Dataset(input_grid_file, 'r')

    ak = ifh_grid.variables['ak']
    bk = ifh_grid.variables['bk']
    zsurf = ifh_grid.variables['zsurf']
  
      
  ifh_data = netCDF4.Dataset(input_data_file, 'r')

#   print(dir(ifh_data.dimensions['lat']))
#   print(ifh_data.dimensions['lat'].size)
#   print(ifh_data.dimensions['lat'].isunlimited)
  
#   raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

  ofh = netCDF4.Dataset(output_file, 'w', format=nc_model)
    
  global_dictionary = {}
  for attr in ifh_data.ncattrs():
    global_dictionary[attr] = getattr(ifh_data,attr)

  dims_dictionary = {}
  for dims in ifh_data.dimensions.keys():
#     print('dims=', dims, ifh_data.dimensions[dims].size, ifh_data.dimensions[dims].isunlimited)
    
#     if(ifh_data.dimensions[dims].isunlimited):
#       print('yes')
    
#currently, convert all time dimensions to unlimited.
    if(dims=='time'): #PLEV.exe seems to need unlimited time dimensions to work.
      dims_dictionary[ifh_data.dimensions[dims].name] = 0
      #dims_dictionary[ifh_data.dimensions[dims].name] = ifh_data.dimensions[dims].size
    else:
      dims_dictionary[ifh_data.dimensions[dims].name] = ifh_data.dimensions[dims].size

  #dims_dictionary['phalf'] = ifh_grid.dimensions['phalf'].size
  
  #print(dims_dictionary)
  
#   phalf_test=True #assume in file.
  ps_test=phalf_test=bk_test=pk_test=zsurf_test=False #assume not in file.
  
#   try:
#     dummy=dims_dictionary['phalf']
#   except KeyError:
#     dims_dictionary['phalf'] = ifh_grid.dimensions['phalf'].size
#     phalf = ifh_grid.variables['phalf']
#     phalf_test=False

  #ofh.close()
  #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))      

  dims_dictionary_out = {}
  for dims in dims_dictionary.keys():
    #print('xxx',dims,dims_dictionary[dims],ifh_data.dimensions[dims].isunlimited)
    
#     if(ifh_data.dimensions[dims].isunlimited):
#       print('aaa')
#     else:
#       print('bbb')

#     if(dims=='time'):
#       dims_size = None
#     else:
#       dims_size = ifh_data.dimensions[dims].size
      
    dims_dictionary_out[dims] = ofh.createDimension(dims, dims_dictionary[dims])
    #dims_dictionary_out[dims] = ofh.createDimension(dims, size=dims_size)
    #dims_dictionary_out[dims] = ofh.createDimension(dims), size=ifh_data.dimensions[dims].isunlimited)
    
#     if(dims=='time'):
#       ofh.sync()
#       ofh.close()
#       raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

#   all_vars=ifh_data.variables
  
#   print(type(all_vars))
  
#   #print(ifh_data.variables.values())

#   pk_test = all_vars.get('pk', None)
  
  #print(pk_test)
  
#   try:
#     dummy=dims_dictionary['phalf']
#   except KeyError:
#     dims_dictionary['phalf'] = ifh_grid.dimensions['phalf'].size
#     phalf = ifh_grid.variables['phalf']
#     phalf_test=False

  #if all_vars.has_key('key1'):
    
  #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
    
  #vars_dictionary_out = {}


  for cnt, var in enumerate(ifh_data.variables.keys()):
    if(Diag): print('cnt,var=',cnt,var)
      
    if(var=='pk'):
      pk_test=True
    elif(var=='bk'):
      bk_test=True
    elif(var=='zsurf'):
      zsurf_test=True
    elif(var=='phalf'):
      phalf_test=True
    elif(var=='ps'):
      ps_test=True   
      
    input_variable = ifh_data.variables[var]
    
    input_variable_atts_tmp=input_variable.ncattrs()
    
    try:
      fill_value_locator=input_variable_atts_tmp.index('_FillValue')
    except ValueError:
      fill_value_locator=-1
    
    #print('fill_value_locator=',fill_value_locator)
    
    if(fill_value_locator>=0):
      fill_value=getattr(input_variable, '_FillValue')
    else:
      fill_value=None
      
    #print('fill_value=',fill_value)
    
    #print('input_variable_atts_tmp',input_variable_atts_tmp)
    
    #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
      
#     dims_dictionary_out[var] = ofh.createVariable(var, \
#                                                     ifh_data.variables[var].datatype, \
#                                                     ifh_data.variables[var].dimensions, \
#                                                     zlib=zlib, \
#                                                     complevel=complevel, fill_value=fill_value)


    dims_dictionary_out[var] = ofh.createVariable(var, \
                                                    ifh_data.variables[var].datatype, \
                                                    ifh_data.variables[var].dimensions, \
                                                    zlib=False, \
                                                    complevel=complevel, fill_value=fill_value)
    
    #print('var_dictionary=',var_dictionary)
    
    var_dictionary = {}
    
    #strip off _FillValue as defined at variable definition time.
    input_variable_atts=[]
    for input_variable_att in input_variable_atts_tmp:
      if(input_variable_att!='_FillValue'):
        input_variable_atts.append(input_variable_att)
        
    #print('input_variable_atts=',input_variable_atts)
    
    for attr in input_variable_atts:
      #print('attr=',attr)
      var_dictionary[attr] = getattr(input_variable, attr)
      
    #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
    
    #print('var_dictionary=',var_dictionary)
    
    dims_dictionary_out[var][:] = input_variable[:]
    
    dims_dictionary_out[var].setncatts(var_dictionary)

    #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  if(not ps_test):
    raise SystemExit('ps missing from data file:'+__file__+' line number: '+str(inspect.stack()[0][2]))
    
  if(type(input_grid_file)!=type(None)):

    if(not pk_test):
      try:
        pk_fill=ifh_grid.variables['ak']._FillValue
      except:
        pk_fill=None
      pk_var = ofh.createVariable('pk', ifh_grid.variables['ak'].datatype, \
                                 ifh_grid.variables['ak'].dimensions, \
                                fill_value=pk_fill \
                                 )
      pk_var[:] = ak[:]
      ak_atts=ak.ncattrs()
      ak_dictionary={}
      for ak_att in ak_atts:
        if(ak_att!='_FillValue'):
          ak_dictionary[ak_att] = getattr(ak, ak_att)
      pk_setatts=pk_var.setncatts(ak_dictionary)
    
    if(not bk_test):
      try:
        bk_fill=ifh_grid.variables['bk']._FillValue
      except:
        bk_fill=None
      bk_var = ofh.createVariable('bk', ifh_grid.variables['bk'].datatype, \
                                 ifh_grid.variables['bk'].dimensions, \
                                fill_value=bk_fill \
                                 )
      bk_var[:] = bk[:]
      bk_atts=bk.ncattrs()
      bk_dictionary={}
      for bk_att in bk_atts:
        if(bk_att!='_FillValue'):
          bk_dictionary[bk_att] = getattr(bk, bk_att)
      bk_setatts=bk_var.setncatts(bk_dictionary)
    
    if(not zsurf_test):
      try:
        zsurf_fill=ifh_grid.variables['zsurf']._FillValue
      except:
        zsurf_fill=None
      zsurf_var = ofh.createVariable('zsurf', ifh_grid.variables['zsurf'].datatype, \
                                 ifh_grid.variables['zsurf'].dimensions, \
                                fill_value=zsurf_fill \
                                 )
      zsurf_var[:] = zsurf[:]
      zsurf_atts=zsurf.ncattrs()
      zsurf_dictionary={}
      for zsurf_att in zsurf_atts:
        if(zsurf_att!='_FillValue'):
          zsurf_dictionary[zsurf_att] = getattr(zsurf, zsurf_att)
      zsurf_setatts=zsurf_var.setncatts(zsurf_dictionary)
    
    if(not phalf_test):
      try:
        phalf_fill=ifh_grid.variables['phalf']._FillValue
      except:
        phalf_fill=None      
      phalf_var = ofh.createVariable('phalf', ifh_grid.variables['phalf'].datatype, \
                               ifh_grid.variables['phalf'].dimensions, \
                              fill_value=phalf_fill \
                               )
      phalf_var[:] = phalf[:]
      
      phalf_atts=phalf.ncattrs()
      phalf_dictionary={}
      for phalf_att in phalf_atts:
        if(phalf_att!='_FillValue'):
          phalf_dictionary[phalf_att] = getattr(phalf, phalf_att)
      phalf_setatts=phalf_var.setncatts(phalf_dictionary)
    
  ofh.sync()
  
  output_file_size=file_size(output_file)
  
  compression=100* (1 - float(output_file_size.split()[0])/float(input_data_file_size.split()[0]))
  
  if(history):
    history_to_append = ' prepare_vertintp.py:'+ \
    ' input_data_file='+input_data_file+ \
    ' input_grid_file='+str(input_grid_file)+ \
    ', output file='+output_file+ \
    ', compressions='+str(compression)+ \
    ', creation time='+str(datetime.datetime.now())+'.'
  
    try:
      history_value=global_dictionary['history']+history_to_append
    except KeyError:
      history_value=history_to_append
    
  #print(history)
  
    global_dictionary['history'] = history_value
  
  ofh.setncatts(global_dictionary)

  print('Output file: '+output_file+', File size='+output_file_size+', Compression (approx.)='+str(compression)+'%.')
    
  ofh.close()

  #if(Diag): print('Total time =',timeit.timeit() - time_start)
  
  return(0) #end of prepare_vertintp

print('END')

END


In [29]:
print('BEGIN')


idir_data='/OSM/CBR/OA_DCFP/data3/CAFEPP/short/v14/tok599/coupled/ao_am2/coupled_da/workdir2/OUTPUT-2step-nobreeding-carbon2/20020101'
ifil_data='atmos_daily_2002_01_01.nc'

idir_data='/OSM/CBR/OA_DCFP/data/CAFEPP/g/data1/v14/coupled_model/v0/OUTPUT'
ifil_data='atmos_month_0500_01.nc'

idir_data='/OSM/CBR/OA_DCFP/data1/model_output/CAFE/forecasts/f1/hybrid/yr2008/mn1/OUTPUT.1'
ifil_data='atmos_daily_2008_01_01.nc'

idir_data='/OSM/CBR/OA_DCFP/data1/CAFEPP/g/data1/v14/coupled_model/v1/OUTPUT'
ifil_data='atmos_daily_0500_01_01.nc'

#idir_data='/OSM/CBR/OA_DCFP/data1/CAFEPP/g/data1/v14/coupled_model/v1/OUTPUT'
idir_data='/OSM/CBR/OA_DCFP/work/col414/CAFEPP/g/data1/v14/coupled_model/v1/OUTPUT'
ifil_data='atmos_month_0500_01.nc'

idir_data='/OSM/CBR/OA_DCFP/work/col414/CAFEPP/short/v19/mac599/ao_am2/oct18a/OUTPUT'
ifil_data='atmos_month_1980_01.nc'

idir_data='/OSM/CBR/OA_DCFP/work/col414/CAFEPP/short/v19/mac599/ao_am2/oct18a/OUTPUT'
ifil_data='atmos_daily_1980_01_01.nc'

idir_grid='/OSM/CBR/OA_DCFP/work/squ027/squire_scratch/projects/cafe_hybrid_to_isobaric/shared'
ifil_grid='cafe_grid_info.nc3.nc'
ifil_grid='cafe_grid_info.nc'

#status = prepare_vertintp(idir_data+'/'+ifil_data, idir_grid+'/'+ifil_grid, './test.nc', Diag=False, nc_model='NETCDF4_CLASSIC', compression=1, Clobber=True, history=True)

#nc_model='NETCDF3_64BIT_OFFSET'
#nc_model='NETCDF3_64BIT_DATA'
#nc_model='NETCDF3_CLASSIC'
nc_model='NETCDF4_CLASSIC'
#nc_model='NETCDF4'

status = prepare_vertintp(idir_data+'/'+ifil_data, \
                          idir_grid+'/'+ifil_grid, \
                          './test4.nc', \
                          Diag=True, \
                          nc_model=nc_model, \
                          compression=1, \
                          Clobber=True, \
                          history=True)
  
if(status!=0):
  raise SystemExit('prepare_vertintp non-zero return status:'+__file__+' line number: '+str(inspect.stack()[0][2]))

status=os.system('nccopy -6 test4.nc test.cpy.nc')

#shutil.copy('test4.nc', 'test.cpy.nc')
            
print('END')

BEGIN
Turning on diagnostics.
Using nc model= NETCDF4_CLASSIC
Compressing with level (b/w 1-9)= 1
Overwriting output file if it exists.
global history attribute being appended to or created.
Input data file: /OSM/CBR/OA_DCFP/work/col414/CAFEPP/short/v19/mac599/ao_am2/oct18a/OUTPUT/atmos_daily_1980_01_01.nc, File size=2.4 GB
Input grid file: /OSM/CBR/OA_DCFP/work/squ027/squire_scratch/projects/cafe_hybrid_to_isobaric/shared/cafe_grid_info.nc, File size=118.1 KB
Output file: ./test4.nc
cnt,var= 0 lon
cnt,var= 1 lonb
cnt,var= 2 lat
cnt,var= 3 latb
cnt,var= 4 time
cnt,var= 5 nv
cnt,var= 6 phalf
cnt,var= 7 pfull
cnt,var= 8 evap
cnt,var= 9 lwflx
cnt,var= 10 shflx
cnt,var= 11 t_ref
cnt,var= 12 q_ref
cnt,var= 13 u_ref
cnt,var= 14 v_ref
cnt,var= 15 t_surf
cnt,var= 16 t_ref_min
cnt,var= 17 t_ref_max
cnt,var= 18 ps
cnt,var= 19 h500
cnt,var= 20 hght
cnt,var= 21 sphum
cnt,var= 22 temp
cnt,var= 23 ucomp
cnt,var= 24 vcomp
cnt,var= 25 precip
cnt,var= 26 lwdn_sfc
cnt,var= 27 olr
cnt,var= 28 swdn_sfc
cn

In [211]:
#print(dims_dictionary)
#status=os.system('nccopy -6 test4.nc test.cpy.nc')

NameError: name 'dims_dictionary' is not defined

In [30]:
print('BEGIN')

#import commands
import sys

os.chdir(rundir)

os.chdir('/OSM/CBR/OA_DCFP/work/col414/cafepp')

print('Current Working Directory=',os.getcwd())

levels = [100, 500, 1000, 2000, 3000, 5000, 7000, 10000, 15000, 20000, 25000, \
 30000, 40000, 50000, 60000, 70000, 85000, 92500, 100000] #cmip6 19 standard pressure levels
                
#levels = [20000, 30000, 40000, 45000, 50000, 55000, 60000, 70000, 85000, 100000]

if(os.path.exists('/OSM/CBR/OA_DCFP/work/col414/cafepp/test2.nc')):
  print('Output file exists, deleting.')
  os.remove('/OSM/CBR/OA_DCFP/work/col414/cafepp/test2.nc')

  #        ' -t 1,10,3' + \
command='/OSM/CBR/OA_DCFP/work/col414/cafepp/plevel_col414.bash ' + \
        ' -a' + \
        ' -i /OSM/CBR/OA_DCFP/work/col414/cafepp/test.cpy.nc' + \
        ' -o /OSM/CBR/OA_DCFP/work/col414/cafepp/test2.nc' + \
        ' -p "' + ' '.join(map(str, levels)) + '"' + \
        ' hght'

status, out, err = get_exitcode_stdout_stderr(command)

#print('status, out, err=',status, out, err)

if(status!=0):
  raise SystemExit('plevel_col414.bash non-zero return status:'+__file__+' line number: '+str(inspect.stack()[0][2]))

idir='/OSM/CBR/OA_DCFP/work/col414/cafepp'
ifil='test2.nc'

status = compress_nc(idir+'/'+ifil, \
                     './test3.nc', \
                     Diag='False', \
                     nc_model='NETCDF4_CLASSIC', \
                     compression=1, \
                     Clobber='True', \
                     history='True')
  
if(status!=0):
  raise SystemExit('compress_nc non-zero return status:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
print('END')

BEGIN
Current Working Directory= /OSM/CBR/OA_DCFP/work/col414/cafepp
status, out, err= 0 b'itime =       1\nitime =       2\nitime =       3\nitime =       4\nitime =       5\nitime =       6\nitime =       7\nitime =       8\nitime =       9\nitime =      10\nitime =      11\nitime =      12\nitime =      13\nitime =      14\nitime =      15\nitime =      16\nitime =      17\nitime =      18\nitime =      19\nitime =      20\nitime =      21\nitime =      22\nitime =      23\nitime =      24\nitime =      25\nitime =      26\nitime =      27\nitime =      28\nitime =      29\nitime =      30\nitime =      31\nitime =      32\nitime =      33\nitime =      34\nitime =      35\nitime =      36\nitime =      37\nitime =      38\nitime =      39\nitime =      40\nitime =      41\nitime =      42\nitime =      43\nitime =      44\nitime =      45\nitime =      46\nitime =      47\nitime =      48\nitime =      49\nitime =      50\nitime =      51\nitime =      52\nitime =      53\nitime = 

In [19]:
print('BEGIN')

idir='/OSM/CBR/OA_DCFP/work/col414/cafepp'
ifil='test2.nc'

status = compress_nc(idir+'/'+ifil, \
                     './test3.nc', \
                     Diag='False', \
                     nc_model='NETCDF4_CLASSIC', \
                     compression=1, \
                     Clobber='True', \
                     history='True')
  
if(status!=0):
  raise SystemExit('compress_nc non-zero return status:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
print('END')

BEGIN
Input file: /OSM/CBR/OA_DCFP/work/col414/cafepp/test2.nc, File size=150.2 MB
Output file: ./test3.nc
Output file: ./test3.nc, File size=99.3 MB, Compression (approx.)=33.88814913448734%.
END


In [170]:
import sys
import shlex
from subprocess import Popen, PIPE

def get_exitcode_stdout_stderr(cmd):
    """
    Execute the external command and get its exitcode, stdout and stderr.
    """
    args = shlex.split(cmd)
 
    proc = Popen(args, stdout=PIPE, stderr=PIPE)
    out, err = proc.communicate()
    exitcode = proc.returncode
    #
    return exitcode, out, err
  
os.chdir('/OSM/CBR/OA_DCFP/work/col414/cafepp')

print('Current Working Directory=',os.getcwd())

#status = subprocess.call('/apps/fre-nctools/20180125/bin/PLEV.exe', shell=True)

#command='/apps/fre-nctools/20180125/bin/PLEV.exe'

#command='/OSM/CBR/OA_DCFP/work/col414/cafepp/plevel_col414.bash -t 1,10,3 -a -i /OSM/CBR/OA_DCFP/data1/col414/atmos_daily_2003_01_01.nc -o /OSM/CBR/OA_DCFP/data1/col414/atmos_daily_2003_01_01.plevel.nc'

command='/OSM/CBR/OA_DCFP/work/col414/cafepp/plevel_col414.bash ' + \
        ' -t 1,10,3' + \
        ' -a' + \
        ' -i /OSM/CBR/OA_DCFP/work/col414/cafepp/test.cpy.nc' + \
        ' -o /OSM/CBR/OA_DCFP/work/col414/cafepp/test2.nc' + \
        ' -p "' + ' '.join(map(str, levels)) + '"' + \
        ' hght'

exitcode, out, err = get_exitcode_stdout_stderr(command)

print('exitcode, out, err=',exitcode, out, err)

print('END')

Current Working Directory= /OSM/CBR/OA_DCFP/work/col414/cafepp
exitcode, out, err= 0 b'hello\nitime =       1\nitime, otime =       4       2\nitime, otime =       7       3\nitime, otime =      10       4\n' b''
END
